# Text Classification Using BART
 Fine Tuning Transformer for MultiLabel Text Classification

Installing Transformers

In [1]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.0 MB/s eta 0:00:00


In [2]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [34]:
import torch
torch.cuda.empty_cache()

## Importing Necessary Libraries

In [9]:
# Importing stock ml libraries
import numpy as np
import pandas as pd
from sklearn import metrics
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import transformers
import requests
from io import BytesIO
import tarfile

from transformers import BartTokenizer, BartForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report




## Extract Data from source

In [4]:
url = "https://s3.amazonaws.com/fast-ai-nlp/amazon_review_full_csv.tgz"
response = requests.get(url)
file = tarfile.open(fileobj=BytesIO(response.content))
file.extractall()

In [37]:
train = pd.read_csv('amazon_review_full_csv/train.csv',usecols=[0,1,2], names=['Rating', 'Title','ReviewText'],nrows=10000)
test = pd.read_csv('amazon_review_full_csv/test.csv',usecols=[0,1,2], names=['Rating', 'Title','ReviewText'],nrows=3500)

In [38]:
train.shape

(10000, 3)

In [39]:
test.shape

(3500, 3)

In [40]:
# Define function to classify ratings
def classify_rating(rating):
    if rating > 3:
        return 'Positive'
    elif rating < 3:
        return 'Negative'
    else:
        return 'Neutral'

In [41]:
train['Sentiment'] = train['Rating'].apply(lambda x: classify_rating(x))
test['Sentiment'] = test['Rating'].apply(lambda x: classify_rating(x))

In [42]:
train,test = train[['Sentiment','ReviewText']], test[['Sentiment','ReviewText']]

## Convert labels to numeric values

In [43]:

label2id = {"Positive": 2, "Neutral": 1, "Negative": 0}
train["label"] = train["Sentiment"].map(label2id)
test["label"] = test["Sentiment"].map(label2id)

In [44]:
train["Sentiment"] = train["label"]
test["Sentiment"] = test["label"]

In [45]:
train,test = train[['Sentiment','ReviewText']], test[['Sentiment','ReviewText']]

In [46]:
train.head()

,Sentiment,ReviewText
0,1,Gave this to my dad for a gag gift after direc...
1,2,I hope a lot of people hear this cd. We need m...
2,2,I'm reading a lot of reviews saying that this ...
3,2,The music of Yasunori Misuda is without questi...
4,2,Probably the greatest soundtrack in history! U...


In [47]:
test.head()

,Sentiment,ReviewText
0,0,"This model may be ok for sedentary types, but ..."
1,2,This is a fast read filled with unexpected hum...
2,0,I bought one of these chargers..the instructio...
3,0,I was excited to find a book ostensibly about ...
4,0,"I am a big JVC fan, but I do not like this mod..."


## Load the BART tokenizer and encode the text:

In [48]:
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base", max_length=512)  # Set max_length to a smaller value

In [49]:
class AmazonReviewDataset(Dataset):
    def __init__(self, reviews, labels):
        self.reviews = reviews
        self.Sentiment = labels

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        review = str(self.reviews[idx])
        label = self.Sentiment[idx]
        encoding = tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=512,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "label": torch.tensor(label, dtype=torch.long)
        }



In [50]:
train_dataset = AmazonReviewDataset(train["ReviewText"], train["Sentiment"])
val_dataset = AmazonReviewDataset(test["ReviewText"], test["Sentiment"])

## Create data loaders:

In [51]:
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)


## Load the pre-trained DistillBERT model for sequence classification:

In [52]:
model = BartForSequenceClassification.from_pretrained("facebook/bart-base", num_labels=len(label2id))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.out_proj.bias', 'classification_head.dense.weight', 'classification_head.dense.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BartForSequenceClassification(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps

## Define the optimizer and learning rate scheduler:

In [53]:
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01)
total_steps = len(train_loader) * 5  # 5 epochs
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)


## Train the model:

In [54]:
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs} - Avg Loss: {avg_loss:.4f}")

    # Validation
    model.eval()
    val_predictions = []
    val_labels = []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            _, predictions = torch.max(logits, dim=1)
            val_predictions.extend(predictions.detach().cpu().numpy())
            val_labels.extend(labels.detach().cpu().numpy())
    val_acc = accuracy_score(val_labels, val_predictions)
    print(f"Validation Accuracy: {val_acc:.4f}")


Epoch 1/5 - Avg Loss: 1.0783
Validation Accuracy: 0.4086
Epoch 2/5 - Avg Loss: 1.0611
Validation Accuracy: 0.4086
Epoch 3/5 - Avg Loss: 1.0611
Validation Accuracy: 0.4086
Epoch 4/5 - Avg Loss: 1.0611
Validation Accuracy: 0.4086
Epoch 5/5 - Avg Loss: 1.0612
Validation Accuracy: 0.4086


## Evaluate the model on training set

In [ ]:
# Evaluation on the train set

model.eval()
train_predictions = []
train_labels = []
with torch.no_grad():
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        _, predictions = torch.max(logits, dim=1)
        train_predictions.extend(predictions.detach().cpu().numpy())
        train_labels.extend(labels.detach().cpu().numpy())

train_acc = accuracy_score(train_labels, train_predictions)
classification_rep = classification_report(train_labels, train_predictions, target_names=label2id.keys())

print(f"Train Accuracy: {train_acc:.4f}")
print("\nClassification Report:")
print(classification_rep)



Test Accuracy: 0.3991

Classification Report:
              precision    recall  f1-score   support

    Positive       0.57      0.03      0.06      4073
     Neutral       0.00      0.00      0.00      2010
    Negative       0.40      0.99      0.56      3917

    accuracy                           0.40     10000
   macro avg       0.32      0.34      0.21     10000
weighted avg       0.39      0.40      0.24     10000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
